In [1]:
%load_ext autoreload
%autoreload 2

In [460]:
import pandas as pd
import jax.numpy as jnp
import glob
import data_loading
import eval

In [461]:
# load sample lobster data
message_paths = glob.glob('data/*message*.csv')
book_paths = glob.glob('data/*orderbook*.csv')

In [462]:
m = data_loading.load_message_df(message_paths[0])
b = data_loading.load_book_df(book_paths[0])

In [9]:
m

,time,event_type,order_id,size,price,direction
0,34200.015105074,4,15818974,4,5794000,1
1,34200.05990197,4,15818974,300,5794000,1
2,34200.113246707,5,0,1,5795100,1
3,34200.113246707,5,0,1,5795000,1
4,34200.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,57599.544131783,1,287157290,300,5655600,-1
147912,57599.544132627,1,287157291,200,5655600,-1
147913,57599.544133264,1,287157292,300,5655600,-1
147914,57599.582154048,3,287027555,100,5649700,1


In [72]:
eval.time_to_first_fill(m).mean()

17.77700116925748

In [71]:
eval.time_to_cancel(m).mean()

7.965471268527362

In [97]:
b

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [80]:
eval.l1_volume(m, b)

,ask_vol,bid_vol,time_weight
0,100,496,NaN
1,100,196,0.000001914408993084087627622518347
2,100,196,0.000002279703581393350855837805738
3,100,196,0
4,100,196,0
...,...,...,...
147911,42,200,8.290649081095105334056372105E-11
147912,42,200,3.606859703321788093785349514E-11
147913,42,200,2.722238899308032009172118057E-11
147914,42,200,0.000001624829783155234450326830761


In [96]:
eval.total_volume(m, b, 1)

,ask_vol,bid_vol,time_weight
0,100,5802300,NaN
1,100,5802300,0.000001914408993084087627622518347
2,100,5802300,0.000002279703581393350855837805738
3,100,5802300,0
4,100,5802300,0
...,...,...,...
147911,42,5652200,8.290649081095105334056372105E-11
147912,42,5652200,3.606859703321788093785349514E-11
147913,42,5652200,2.722238899308032009172118057E-11
147914,42,5652200,0.000001624829783155234450326830761


In [115]:
eval.limit_order_depth(m, b)

(10        2450.0
 12        3050.0
 14        3350.0
 16        3550.0
 18        3550.0
            ...  
 147909    3900.0
 147910    3900.0
 147911    3900.0
 147912    3900.0
 147913    3900.0
 Length: 37136, dtype: float64,
 7         4150.0
 8         6950.0
 9         8150.0
 11        6450.0
 13        6550.0
            ...  
 147889    3950.0
 147891    2600.0
 147892    3700.0
 147894    3200.0
 147896    4000.0
 Length: 34122, dtype: float64)

In [116]:
eval.cancellation_depth(m, b)

(45         9250.0
 47        11650.0
 76         3050.0
 77         2750.0
 81         3050.0
            ...   
 147876     2850.0
 147897     3300.0
 147898     2200.0
 147904     3900.0
 147915     3300.0
 Length: 33222, dtype: float64,
 44        25650.0
 46        23250.0
 69         7550.0
 70        14550.0
 71         9550.0
            ...   
 147883     3800.0
 147887     4000.0
 147893     2600.0
 147903     1100.0
 147914     2000.0
 Length: 31724, dtype: float64)

In [125]:
eval._order_levels(m, b, (1,2,3))

(3          1
 39         1
 42         1
 43         1
 47         1
           ..
 136005    10
 136019    10
 136025    10
 136113    10
 136217    10
 Length: 49808, dtype: int64,
 0          1
 40         1
 44         1
 64         1
 65         1
           ..
 135877    10
 136010    10
 136030    10
 136053    10
 136194    10
 Length: 45838, dtype: int64)

In [127]:
eval.cancel_order_levels(m, b)

(8         1
 10        1
 12        1
 47        1
 52        1
          ..
 63101    10
 63371    10
 63406    10
 63420    10
 64446    10
 Length: 12672, dtype: int64,
 73        1
 77        1
 108       1
 124       1
 352       1
          ..
 63942    10
 64079    10
 64143    10
 64264    10
 64560    10
 Length: 11716, dtype: int64)

In [129]:
b

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [128]:
eval.spread(b)

0         8300
1         8300
2         8300
3         8300
4         8300
          ... 
147911    1000
147912    1000
147913    1000
147914    1000
147915    1000
Length: 147916, dtype: int64

In [188]:
ret = eval.mid_returns(m, b, '1min')
ret

time
0 days 09:31:00         NaN
0 days 09:32:00    0.000701
0 days 09:33:00   -0.001393
0 days 09:34:00    0.000329
0 days 09:35:00    0.000251
                     ...   
0 days 15:56:00    0.000390
0 days 15:57:00    0.000425
0 days 15:58:00   -0.000354
0 days 15:59:00    0.001275
0 days 16:00:00    0.000416
Length: 390, dtype: float64

In [189]:
eval.time_to_first_fill(m)

order_id
858821       12.002251
2175511      12.002251
2176461      12.002898
4496151      11.939107
4773351      11.969646
               ...    
287076073     0.102907
287081956     0.000316
287081969     0.000234
287091827     0.275303
287107097     0.114062
Length: 5480, dtype: float64

In [174]:
eval.autocorr(ret)

array([-0.13855764])

In [182]:
eval.autocorr(ret**2, 10, alpha=None)

array([ 1.00000000e+00,  3.97760808e-01,  1.46716661e-01,  1.21313544e-01,
        3.83928983e-02, -1.50425941e-02,  6.71807872e-02,  1.02656084e-01,
        6.61179836e-02,  3.98363877e-04, -2.46290823e-02])

In [183]:
eval.l1_volume(m, b)

,ask_vol,bid_vol,time_weight
0,100,496,NaN
1,100,196,0.000001914408993084087627622518347
2,100,196,0.000002279703581393350855837805738
3,100,196,0
4,100,196,0
...,...,...,...
147911,42,200,8.290649081095105334056372105E-11
147912,42,200,3.606859703321788093785349514E-11
147913,42,200,2.722238899308032009172118057E-11
147914,42,200,0.000001624829783155234450326830761


In [194]:
eval.total_volume(m, b, 2)

,ask_vol,bid_vol,time_weight
time,,,
0 days 09:30:00.015105074,596,11596300,NaN
0 days 09:30:00.059901970,296,11596300,0.000001914408993084087627622518347
0 days 09:30:00.113246707,296,11596300,0.000002279703581393350855837805738
0 days 09:30:00.113246707,296,11596300,0
0 days 09:30:00.113246707,296,11596300,0
...,...,...,...
0 days 15:59:59.544131783,242,11303400,8.290649081095105334056372105E-11
0 days 15:59:59.544132627,242,11303400,3.606859703321788093785349514E-11
0 days 15:59:59.544133264,242,11303400,2.722238899308032009172118057E-11


In [ ]:
# TODO: how to calculate by time?
#       should be able to do this by calculating on whole day data and then grouping
#       or by calculating for shorter time periods and then grouping and aggregating

In [490]:
loader = data_loading.Simple_Loader('data_test_real', 'data_test_gen')

In [491]:
m_real, b_real, m_gen, b_gen = loader[0]

In [396]:
for i in loader:
    print(i)

(                                time  event_type   order_id  size    price  \
0      2012-06-21 09:30:00.015105074           4   15818974     4  5794000   
1      2012-06-21 09:30:00.005990197           4   15818974   300  5794000   
2      2012-06-21 09:30:00.113246707           5          0     1  5795100   
3      2012-06-21 09:30:00.113246707           5          0     1  5795000   
4      2012-06-21 09:30:00.113246707           5          0     1  5794900   
...                              ...         ...        ...   ...      ...   
147911 2012-06-21 15:59:59.544131783           1  287157290   300  5655600   
147912 2012-06-21 15:59:59.544132627           1  287157291   200  5655600   
147913 2012-06-21 15:59:59.544133264           1  287157292   300  5655600   
147914 2012-06-21 15:59:59.582154048           3  287027555   100  5649700   
147915 2012-06-21 15:59:59.871751084           3  287151822   100  5655000   

        direction  
0               1  
1               1  
2 

In [ ]:
ask_lvls, bid_lvls = eval.limit_order_levels(m_real, b_real)

order_prices (71258,)
level_prices (71258, 20)
(71258, 2)


In [ ]:
ask_lvls

time
2012-06-21 09:30:00.202322801    1
2012-06-21 09:30:00.202322801    2
2012-06-21 09:30:00.202322801    3
2012-06-21 09:30:00.202322801    4
2012-06-21 09:30:00.202322801    4
                                ..
2012-06-21 15:59:59.544126313    3
2012-06-21 15:59:59.544129843    3
2012-06-21 15:59:59.544131783    3
2012-06-21 15:59:59.544132627    3
2012-06-21 15:59:59.544133264    3
Length: 37136, dtype: int64

In [476]:
m_real

,time,event_type,order_id,size,price,direction
0,34200.015105074,4,15818974,4,5794000,1
1,34200.05990197,4,15818974,300,5794000,1
2,34200.113246707,5,0,1,5795100,1
3,34200.113246707,5,0,1,5795000,1
4,34200.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,57599.544131783,1,287157290,300,5655600,-1
147912,57599.544132627,1,287157291,200,5655600,-1
147913,57599.544133264,1,287157292,300,5655600,-1
147914,57599.582154048,3,287027555,100,5649700,1


In [466]:
b_real

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [527]:
eval.mid_returns(m_real, b_real)

time
2012-06-21 09:31:00         NaN
2012-06-21 09:32:00    0.000701
2012-06-21 09:33:00   -0.001393
2012-06-21 09:34:00    0.000329
2012-06-21 09:35:00    0.000251
                         ...   
2012-06-21 15:56:00    0.000390
2012-06-21 15:57:00    0.000425
2012-06-21 15:58:00   -0.000354
2012-06-21 15:59:00    0.001275
2012-06-21 16:00:00    0.000416
Freq: min, Length: 390, dtype: float64

In [477]:
m_real.loc[m_real.event_type == 3].price.max()

5809400

In [478]:
b_real.loc[m_real.loc[m_real.event_type == 3].index][38].max()

5772400

In [ ]:
# ISSUE: some delete (type 3) messages are not in the book data --> outside levels but still in messages???

In [510]:
eval.autocorr(eval.mid_returns(m_real, b_real, '1min'), 4)
# eval.mid_returns(m_real, b_real, '5min')

array([ 1.        , -0.13855764, -0.02199417,  0.00926521,  0.00570809])

In [432]:
b_real.loc[m_real.loc[m_real.event_type == 3].index]

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
44,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5805000,100,5771800,100,5806300,100,5766000,100
45,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5771800,100,5806700,100,5766000,100
46,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5766000,100,5806700,100,5762600,100
47,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5766000,100,5809600,50,5762600,100
69,5797900,210,5791200,100,5798900,25,5790000,41,5799500,50,...,5775100,110,5805000,100,5774500,3,5806300,100,5772400,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147898,5655600,200,5651200,200,5655700,100,5651100,100,5656000,100,...,5649600,100,5657700,800,5649400,100,5657800,100,5649200,300
147903,5652200,42,5651200,200,5655000,200,5651100,100,5655600,200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147904,5652200,42,5651200,200,5655000,200,5651100,100,5655600,100,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [497]:
eval._order_levels(m_real, b_real, (3,))

order_prices (64961,)
level_prices (64961, 20)
(24369, 2)


AssertionError: Not all order prices found in book. (24369 != 64961)

In [415]:
eval.cancel_order_levels(m_real, b_real)

order_prices (64980,)
level_prices (64980, 20)
(24388, 2)


AssertionError: Not all order prices found in book. (24388 != 64980)

In [409]:
# how to handle multiple days? Should we include a date column in the data? Use datetime index instead of timedelta?

In [ ]:
# provide loader fn: takes index -> returns date, real messages, real book, gen messages, (gen book)

# TODO: set datetime index from the beginning using the date provided and the time column
#       